## DQN概述
强化学习逐渐引起公众的注意要归功于谷歌DeepMind公司。DeepMind公司最初是由Demis Hassabis, Shane Legg和Mustafa Suleyman于2010年创立的。DeepMind的成功离不开近几年取得突破进展的深度学习技术。DQN就是DeepMind发表在《Nature》上的第一篇论文。题目是：《Human-level control through deep reinforcement learning》。
## Q-learning
Q-learning方法是异策略时间差分方法：就是更新策略和评估策略是不同的策略，在Q-learning中选择策略是$\epsilon-greedy$策略，而评估策略采用的是贪婪策略，具体思路如下：![](https://pic2.zhimg.com/50/v2-08ab664521ca4d88c4f30464d234e3b5_hd.jpg)
而相应的使用$\epsilon-greedy$策略选择动作的策略如下：![](http://www.zhihu.com/equation?tex=%5C%5B+%5Cpi%5Cleft%28a%7Cs%5Cright%29%5Cgets%5Cleft%5C%7B%5Cbegin%7Barray%7D%7Bc%7D+1-%5Cvarepsilon+%2B%5Cfrac%7B%5Cvarepsilon%7D%7B%5Cleft%7C+A%5Cleft%28s%5Cright%29%5Cright%7C%7D%5C+if%5C+a%3Darg%5Cmax_aQ%5Cleft%28s%2Ca%5Cright%29%5C%5C%5C%5C+%5Cfrac%7B%5Cvarepsilon%7D%7B%5Cleft%7C+A%5Cleft%28s%5Cright%29%5Cright%7C%7D%5C+if%5C+a%5Cne+arg%5Cmax_aQ%5Cleft%28s%2Ca%5Cright%29%5C%5C+%5Cend%7Barray%7D%5Cright.%5C%5C%5C%5C%5Cleft%28policy%5Cright%29+%5C%5D)

## DQN
DQN对Q-learning的修改主要体现在以下三个方面：
* DQN利用深度卷积神经网络逼近值函数
* DQN利用了经验回放对强化学习的学习过程进行训练
* DQN独立设置了目标网络来单独处理时间差分算法中的TD偏差

###  利用深度卷积网络逼近值函数
之前值函数逼近使用的方法都是线性逼近，这里利用的是神经网络实现非线性逼近，用公式表示的话是Q(s,a,$\theta$)，注意这里对值函数进行更新时其实就是更新$\theta$,DQN所使用的神经网络结构是3个卷积层和2个全连接层。

### 使用经验回放来进行训练
之前的深度强化学习很难使网络收敛稳定，但是使用经验回放就可以减少不稳定性，这是因为对神经网络训练时，存在的假设使独立同分布，而通过强化学习采集到的数据之间有数据关联，经验回放可以打破关联性。具体是使用均匀随机采样的方法从数据库中提取数据，利用提取的数据进行训练。

### 设置目标网络来单独处理TD偏差
传统的Q-learning算法每一步更新的知识当前状态的q值，利用神经网络对值函数进行逼近时，每一步更新的是参数$\theta$，利用梯度下降法得到的公式为：$$\theta_{t+1} = \theta_t + \alpha(r+\gamma \max_{a^、}(s^、,a^、,\theta)-Q(s,a,\theta))\triangledown Q(s,a,\theta) $$
其中$r+\gamma \max_{a^、}(s^、,a^、,\theta)$为TD目标，但是其跟$\theta$也有关系。

这里我们称计算TD的网络称为TD网络，以往的神经网络逼近值函数时，计算TD目标动作值函数所用网络参数和梯度计算中所用的网络参数相同，这样容易存在相关性，因此DeepMind将计算TD网络的参数表示为$\theta^-$，计算函数逼近的参数表示为$\theta$，其中前者每一步都更新，后者每隔固定的步数进行更新。因此公式变为：$$\theta_{t+1} = \theta_t + \alpha(r+\gamma \max_{a^、}(s^、,a^、,\theta^-)-Q(s,a,\theta))\triangledown Q(s,a,\theta) $$

## DQN实现
1. 初始化回放记忆D，可容纳数据条数为N
2. 利用随机权值$\theta$来初始化动作-行为值函数Q，输入s,a，输出Q值
3. 令$\theta^- = \theta$初始化计算TD目标的行为动作值Q1
4. 进行M轮循环：
    * 初始化一个状态s1,并且将这个s1转化为特征$\phi1$
    * 从这个状态开始与环境交互：
        1. 选择动作：使用$\epsilon$-greedy策略来选择：若随机概率小于$\epsilon$，则随机选择动作，否则选择$a_t = argmax_a Q(\phi(s_t),a,\theta)$
        2. 执行动作：执行所选择的动作，得到奖励和下个状态$r_t,s_{t+1},\phi(t+1)$
        3. 保存记忆池：将转换($\phi_t,a_t,r_t,\phi_{t+1}$)保存到记忆池当中
        4. 从D中随机抽取minibatch个转换($\phi_j,a_j,r_j,\phi_{j+1}$)
        5. 判断(j+1)步是不是终止状态，如果是$y_{j} = r_{j}$，否则$y_j = r_j+\gamma \max_aQ1(\phi_{j+1},a,\theta^-)$
        6. 按照$((\phi_j,a_j),y_j)$执行一次梯度下降训练
        7. 每隔C步更新一次$\theta^-$
        8. 下一个状态
    * 下一轮循环

## DQN各种改进
### NIPS 2013版本
![](https://pic1.zhimg.com/c24454f472843ef5caef2733d50aba00_r.jpg)
和上述相比：
    * 就一个Q网络来更新，没有使用TargetQ
    * 跟基础的Q学习相比加入了经验池的概念
### Nature 2015版本
就是上述版本：
    * 两个Q网络，使用TargetQ
    * 加入经验池也就是Experience Replay记忆回放的概念
效果提升很明显：![](https://pic4.zhimg.com/50/bc99f61adeb19a7343d70ce6015e303f_hd.jpg)

### 三大改进
![](https://pic2.zhimg.com/50/37e899a561b4e7fcfee04fc75744f319_hd.jpg)
* Double DQN：目的是减少因为max Q值计算带来的计算偏差，或者称为过度估计（over estimation）问题，用当前的Q网络来选择动作，用目标Q网络来计算目标Q，用当前网络选出最大的Q值对应的a，然后使用这个a和目标Q网络来计算Q，可以减少过估计。
* Prioritised replay：也就是优先经验的意思。优先级采用目标Q值与当前Q值的差值来表示。优先级高，那么在经验池中的采样概率就高。
* Dueling Network：将Q网络分成两个通道，一个输出V，一个输出A，最后再合起来得到Q。这个方法主要是idea很简单但是很难想到，然后效果一级棒，因此也成为了ICML的best paper。

## 参考
[DQN概述](https://zhuanlan.zhihu.com/p/26052182)  
[DQN简介](https://zhuanlan.zhihu.com/p/21421729?refer=intelligentunit)  
[DQN改进](https://zhuanlan.zhihu.com/p/21547911)